In [ ]:
# !pip install update xarray 
# !pip install cmocean 
# ! pip install netcdf4
# ! pip install cftime
# ! pip install hdf5
# ! conda install cmocean --y
# ! pip install cartopy 
# ! pip install cmocean
# !apt-get -qq install update python-cartopy python3-cartopy
# !  pip install nc-time-axis
# ! pip install wget
# ! pip install matplotlib
# ! pip install intake
# ! pip  install zarr
# ! pip install dask
# ! pip install tqdm
# ! pip install nc_time_axis
# ! pip install cmip6_preprocessing
# ! pip install gcsfs
# !apt-get -qq install python-cartopy python3-cartopy
# import cartopy

!python -m pip install intake-esm



In [ ]:
%matplotlib inline
# import numpy as np
# import gcsfs
# import pandas as pd
# import xarray as xr
# import warnings
# import matplotlib.pyplot as plt
import intake_esm
# import cartopy
# from cartopy import util
# import cartopy.crs as ccrs
# import zarr
# import dask.array as da  
# from dask.diagnostics import ProgressBar
# from tqdm.autonotebook import tqdm
# import nc_time_axis
# import time

# import cmip6_preprocessing
# from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
#                                AutoMinorLocator)

In [ ]:
from cmip6_preprocessing.preprocessing import combined_preprocessing, replace_x_y_nominal_lat_lon, rename_cmip6

def wrapper(ds):
    ds = ds.copy()
    #ds = rename_cmip6(ds)
    #ds = replace_x_y_nominal_lat_lon(ds)
    
    if ('longitude' in ds.dims):
        ds = ds.rename({'longitude':'lon'}) # some models labelled dimensions differently...
    if ('latitude' in ds.dims):    
        ds = ds.rename({'latitude': 'lat'})

    #------------experimental---------- 
    
    if ('nav_lon' in ds.dims):
        ds = ds.rename({'nav_lon':'lon'}) # some models labelled dimensions differently...
    if ('nav_lat' in ds.dims):    
        ds = ds.rename({'nav_lat': 'lat'})
    if ('vertices' in ds.dims): 
        ds=ds.drop_dims('vertices')   
    if ('nvertex' in ds.dims): 
        ds=ds.drop_dims('nvertex') 
    if ('axis_nbounds' in ds.dims): 
        ds=ds.drop_dims('axis_nbounds')       
    #-----------------------------------    
        
    if ('bnds' in ds.dims): 
        ds=ds.drop_dims('bnds')
    if ('vertex' in ds.dims): 
        ds=ds.drop_dims('vertex')
    if ('height' in ds.dims): 
        ds=ds.drop_dims('height')
    if ('height' in ds): 
        ds=ds.drop_vars('height')
    if ('depth' in ds): 
        ds=ds.drop_vars('depth')
    return ds


def addCyclicPoint(xarray_obj, dim, period=None):
    """
    Add a cyclic longitude point for data that doesn't wrap back to zero. Avoids seam at 0 or 180 longitude
    """
    if period is None:
        period = xarray_obj.sizes[dim] * xarray_obj.coords[dim][:2].diff(dim).item()
    first_point = xarray_obj.isel({dim: slice(1)})
    first_point.coords[dim] = first_point.coords[dim]+period
                                    
    return xr.concat([xarray_obj, first_point], dim=dim)


ds_out = xe.util.grid_global(1, 1)



In [ ]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')


variable_id = 'fgco2' #@param {type:"string"}
table_id = 'Omon' #@param {type:"string"}
member_id = 'r1i1p1f1'

dset_dicts={}  #==dset_dicts_match 
experiment_ids=['historical','ssp126', 'ssp245', 'ssp585']

# this only needs to be created once
gcs = gcsfs.GCSFileSystem(token='anon')

for exp in experiment_ids:
    model_dict={}   #==dset_dicts_proc 
    canesm5_query = df.query(f"table_id == '{table_id}' & "\
                            f"variable_id == '{variable_id}' & "\
                            f"experiment_id == '{exp}' & "\
                            f"member_id == '{member_id}'")
                            


    for zstore in canesm5_query.zstore.values:
        
        model = zstore.split('/')[5]
        mapper = gcs.get_mapper(zstore)
        temp = xr.open_zarr(mapper, consolidated=True)
        ds = wrapper(temp)
        
        try:
            ds = ds.rename({'longitude':'lon', 'latitude':'lat'}) 
              
        except:     
            pass
        
        try:
            ds = ds.rename({'nav_lat':'lat', 'nav_lon':'lon'})
            
        except:
            pass
            
        try:
            regridder = xe.Regridder(ds, ds_out, 'bilinear', ignore_degenerate=True, reuse_weights=True, periodic=True)
            temp = regridder(ds.fgco2)
        except:
            print('Failed:', model)
            temp = ds
        
        model_dict[model] = temp

    dset_dicts[exp] = model_dict

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
dset_dicts

{'historical': {'ACCESS-ESM1-5': <xarray.Dataset>
  Dimensions:             (i: 360, j: 300, time: 1980, vertices: 4)
  Coordinates:
      depth               float64 ...
    * i                   (i) int32 0 1 2 3 4 5 6 ... 353 354 355 356 357 358 359
    * j                   (j) int32 0 1 2 3 4 5 6 ... 293 294 295 296 297 298 299
      latitude            (j, i) float64 dask.array<chunksize=(300, 360), meta=np.ndarray>
      longitude           (j, i) float64 dask.array<chunksize=(300, 360), meta=np.ndarray>
    * time                (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T12:00:00
  Dimensions without coordinates: vertices
  Data variables:
      fgco2               (time, j, i) float32 dask.array<chunksize=(189, 300, 360), meta=np.ndarray>
      vertices_latitude   (j, i, vertices) float64 dask.array<chunksize=(300, 360, 4), meta=np.ndarray>
      vertices_longitude  (j, i, vertices) float64 dask.array<chunksize=(300, 360, 4), meta=np.ndarray>
  Attributes:
      

In [ ]:
# ValueError: destination buffer too small; expected at least 838080, got 419040

for exp in dset_dicts.keys():
    del dset_dicts[exp]['CanESM5']

In [ ]:
# keep only models with historical run
dset_dicts_match = {}
dset_dicts_match['historical'] = dset_dicts['historical']
    
experiment_ids=['ssp126', 'ssp245', 'ssp585']

for exp in experiment_ids:
    tmp = {}
    for name, data in dset_dicts[exp].items():
        if name in dset_dicts['historical'].keys(): 
            tmp[name] = data
    dset_dicts_match[exp] = tmp         

In [ ]:
fgco2_anom_dicts = {}
for exp, dic in dset_dicts_match.items():
    tmp = {}
    if exp == 'historical':
          for model, data in dic.items():
              try:
                  tmp[model] = -abs(data.sel(time=slice('1980','2015')).mean(dim='time') - dset_dicts_match['historical'][model].sel(time=slice('1850','1900')).mean(dim='time'))
                               
              except:
                  print('Failed:', model)
    else:

          for model, data in dic.items():
              try:
                  tmp[model] = -abs(data.sel(time=slice('2015','2050')).mean(dim='time') - dset_dicts_match['historical'][model].sel(time=slice('1980','2015')).mean(dim='time'))
                               
              except:
                  print('Failed:', model)

    fgco2_anom_dicts[exp] = tmp 

In [ ]:
fgco2_anoms = {}
for exp, dic in fgco2_anom_dicts.items():
    dss = [ds for key, ds in dic.items()]
    fgco2_anoms[exp] = xr.concat(dss, dim='source_id').assign_coords(source_id=list(dic.keys())).load()

In [ ]:
multimodel_historical_anomaly = fgco2_anoms['historical'].to_netcdf('multimodel_historical_anomaly')
multimodel_ssp126_anomaly = fgco2_anoms['ssp126'].to_netcdf('multimodel_ssp126_anomaly')
multimodel_ssp245_anomaly = fgco2_anoms['ssp245'].to_netcdf('multimodel_ssp245_anomaly')
multimodel_ssp585_anomaly = fgco2_anoms['ssp585'].to_netcdf('multimodel_ssp585_anomaly')
    

In [ ]:
import wget
file = wget.download('')
obs = xr.open_dataset(file)
obs = wrapper(obs)

# define a regridder from the target to source grid
regridder = xe.Regridder(obs, ds_out, 'bilinear', ignore_degenerate=True, reuse_weights=True, periodic=True)

# Do the actual remapping
fgco2_obs = regridder(obs)



In [ ]:
sec_2_yr = 60*60*24*365
kg_2_mol = 1000./12.
kgs_2_pgyr = sec_2_yr / 1e12

In [ ]:
fig= plt.figure(figsize=(25,16))

ax = plt.subplot(2,2,1)
cb= (fgco2_anoms['historical'].mean(dim='source_id')*kg_2_mol*sec_2_yr).plot()
#  _=ax.coastlines()
ax.set_title('Multi Model mean flux anomaly 1980-2015 to 1850-1950 mean')
# divider = make_axes_locatable(ax)
# ax_cb = divider.append_axes('bottom', size="3%", pad=0.1, axes_class=plt.Axes)
# _=plt.colorbar(cb, cax=ax_cb, orientation="horizontal", label=r'Ocean carbon flux (mol m$^{-2}$ yr$^{-1}$)')


ax = plt.subplot(2,2,2)
cb= (fgco2_anoms['ssp126'].mean(dim='source_id')*kg_2_mol*sec_2_yr).plot()
ax.set_title('Multi Model mean flux anomaly 2015-2050 ssp126 to 1980-2015 mean')
#  _=ax.coastlines()
# divider = make_axes_locatable(ax)
# ax_cb = divider.append_axes('bottom', size="3%", pad=0.1, axes_class=plt.Axes)
# _=plt.colorbar(cb, cax=ax_cb, orientation="horizontal", label=r'Ocean carbon flux (mol m$^{-2}$ yr$^{-1}$)')



ax = plt.subplot(2,2,3)
cb= (fgco2_anoms['ssp245'].mean(dim='source_id')*kg_2_mol*sec_2_yr).plot()
ax.set_title('Multi Model mean flux anomaly 2015-2050 ssp245 to 1980-2015 mean')
#  _=ax.coastlines()
# divider = make_axes_locatable(ax)
# ax_cb = divider.append_axes('bottom', size="3%", pad=0.1, axes_class=plt.Axes)
# _=plt.colorbar(cb, cax=ax_cb, orientation="horizontal", label=r'Ocean carbon flux (mol m$^{-2}$ yr$^{-1}$)')


ax = plt.subplot(2,2,4)
cb= (fgco2_anoms['ssp585'].mean(dim='source_id')*kg_2_mol*sec_2_yr).plot()
ax.set_title('Multi Model mean flux anomaly 2015-2050 ssp585 to 1980-2015 mean')
#  _=ax.coastlines()
# divider = make_axes_locatable(ax)
# ax_cb = divider.append_axes('bottom', size="3%", pad=0.1, axes_class=plt.Axes)
# _=plt.colorbar(cb, cax=ax_cb, orientation="horizontal", label=r'Ocean carbon flux (mol m$^{-2}$ yr$^{-1}$)')